In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
import tensorflow.keras as keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical


In [ ]:
# загрузка тренировочных и тестовых данных
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# конвертация чисел из uint8 в float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# нормализация данных [0, 1]
x_train /= 255 
x_test /= 255 

# трансформация лейблов в one-hot encoding
y_train = to_categorical(y_train, 10) 
y_test = to_categorical(y_test, 10) 

# изменение размерности массива в 4D массив
x_train = x_train.reshape(x_train.shape[0], 28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28,28,1)

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.models import Model 
# инициализация  модели
input1= layers.Input(shape=(28,28,1))
# первый сверточный слой
x1 = layers.Conv2D(6, kernel_size=(7, 7), strides=(1, 1), activation='tanh',  padding="same")(input1)

# второй пуллинговый слой
x2 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x1)

# третий сверточный слой
x3 = layers.Conv2D(16, kernel_size=(7, 7), strides=(1, 1), activation='tanh', padding='valid')(x2)

# четвертый пуллинговый слой
x4 = layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x3)

# пятый  слой
x5 = layers.Conv2D(120, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding='valid')(x4)

# сглаживание CNN выхода чтобы можно было его присоединить к полносвязногому слою
x12 = layers.Flatten()(x5)

# шестой полносвязный слой
x13 = layers.Dense(84, activation='tanh')(x12)

# выходной слой с функцией активации softmax
out_x = layers.Dense(10, activation='softmax')(x13)


# Соберем полную модель сети от входа к выходу 
model1 = Model(inputs = input1, outputs = out_x) 
# сделаем несколько промежуточных выходов (через них посмотрим , что происходит в сети) 
model3 = Model(inputs = input1, outputs = x3)
model5 = Model(inputs = input1, outputs = x5)
# компилияция модели
model1.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

# Обучаем модель
hist = model1.fit(x=x_train,y=y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test), verbose=1)

test_score = model1.evaluate(x_test, y_test)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

Epoch 1/5
469/469 [==============================] - 46s 96ms/step - loss: 1.3145 - accuracy: 0.5788 - val_loss: 0.8646 - val_accuracy: 0.6941
Epoch 2/5
469/469 [==============================] - 44s 95ms/step - loss: 0.7595 - accuracy: 0.7233 - val_loss: 0.7218 - val_accuracy: 0.7305
Epoch 3/5
469/469 [==============================] - 44s 95ms/step - loss: 0.6697 - accuracy: 0.7493 - val_loss: 0.6680 - val_accuracy: 0.7531
Epoch 4/5
469/469 [==============================] - 44s 94ms/step - loss: 0.6229 - accuracy: 0.7687 - val_loss: 0.6282 - val_accuracy: 0.7673
Epoch 5/5
313/313 [==============================] - 5s 16ms/step - loss: 0.6046 - accuracy: 0.7683
Test loss 0.6046, accuracy 76.83%


accurcy - 82.8 для базового варианта

ядро - 3, +1 набор слоёв - 78.73

для всех падднигов 1 на 1 - точность 83.05

добавляем ещё слои - 82.37

сократили слои и расширили ядро до 7 на 7 - 85.02 (3 свёрточных, страйд - 1)

прошлый вариант - страйд 2 - 76.83


Вообщем разумно делать неглубокую сеть с большими ядрами.